<a href="https://colab.research.google.com/github/anydevil0812/BoxOffice/blob/master/%EA%B5%AD%EB%82%B4%20%EB%B0%95%EC%8A%A4%EC%98%A4%ED%94%BC%EC%8A%A4%20%ED%99%95%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# 국내 박스오피스 순위 생성 파일
import re
from bs4 import BeautifulSoup
import requests
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from datetime import date, timedelta
import warnings
warnings.filterwarnings(action='ignore')

# 5일간의 날짜 목록
date_list = []
for i in range(1,6):
    date_list.append(date.today()-timedelta(i))
date_list.sort()

# OPEN API 파라미터 입력을 위한 datetime -> str
date_list = list(map(str, date_list))
date_keyword = []
for i in range(len(date_list)):
    date_keyword.append(date_list[i].replace('-', ''))

keywords = ['rank', 'movieNm', 'openDt', 'audiCnt', 'audiAcc']

# 영화진흥위원회_영화박스오피스 DB OPEN API 이용하여 크롤링
def crawling_boxoffice(date, keyword, purpose):

    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.xml'
    params ={'key' : 'ee0e8dd4b1074b2a1378799562c3d370', "targetDt" : date}
    response = requests.get(url, params=params) # UTF-8 코드 형식로 데이터가 불러와짐
    soup = BeautifulSoup(response.content, 'xml')

    factor = soup.select(keyword)
    factor = str(factor)

    if keyword == 'rank': # 순위
        rule = re.compile('(?<=\<rank>)(.*?)(?=<\/rank>)')
        final = rule.findall(factor)
        for i in range(len(final)):
            final[i] += "위"
        return final

    if keyword == 'movieNm': # 영화명
        rule = re.compile('(?<=\<movieNm>)(.*?)(?=<\/movieNm>)')
        final = rule.findall(factor)
        return final

    if keyword == 'openDt': # 개봉일
        rule = re.compile('(?<=\<openDt>)(.*?)(?=<\/openDt>)')
        final = rule.findall(factor)
        return final

    if keyword == 'audiCnt': # 일일 관객수
        rule = re.compile('(?<=\<audiCnt>)(.*?)(?=<\/audiCnt>)')
        final = rule.findall(factor)
        if purpose == "table":
          for i in range(len(final)):
              final[i] = f'{int(final[i]):,}명'
        if purpose == "graph":
          final = list(map(int, final))      
        return final

    if keyword == 'audiAcc': # 누적 관객수
        rule = re.compile('(?<=\<audiAcc>)(.*?)(?=<\/audiAcc>)')
        final = rule.findall(factor)
        if purpose == "table":
          for i in range(len(final)):
              final[i] = f'{int(final[i]):,}명'
        if purpose == "graph":
          final = list(map(int, final))
        return final

In [17]:
# 테이블 확인용 (관객수 데이터 string)
table_data1 = {"순위" : crawling_boxoffice(date_keyword[0], 'rank', 'table'), "영화명" : crawling_boxoffice(date_keyword[0], 'movieNm', 'table'),
        "개봉일" : crawling_boxoffice(date_keyword[0], 'openDt', 'table'), "일일 관객수" : crawling_boxoffice(date_keyword[0], 'audiCnt', 'table'),
        "누적 관객수" : crawling_boxoffice(date_keyword[0], 'audiAcc', 'table'), "기준 날짜" : date_list[0]}

table_data2 = {"순위" : crawling_boxoffice(date_keyword[1], 'rank', 'table'), "영화명" : crawling_boxoffice(date_keyword[1], 'movieNm', 'table'),
        "개봉일" : crawling_boxoffice(date_keyword[1], 'openDt', 'table'), "일일 관객수" : crawling_boxoffice(date_keyword[1], 'audiCnt', 'table'),
        "누적 관객수" : crawling_boxoffice(date_keyword[1], 'audiAcc', 'table'), "기준 날짜" : date_list[1]}

table_data3 = {"순위" : crawling_boxoffice(date_keyword[2], 'rank', 'table'), "영화명" : crawling_boxoffice(date_keyword[2], 'movieNm', 'table'),
        "개봉일" : crawling_boxoffice(date_keyword[2], 'openDt', 'table'), "일일 관객수" : crawling_boxoffice(date_keyword[2], 'audiCnt', 'table'),
        "누적 관객수" : crawling_boxoffice(date_keyword[2], 'audiAcc', 'table'), "기준 날짜" : date_list[2]}

table_data4 = {"순위" : crawling_boxoffice(date_keyword[3], 'rank', 'table'), "영화명" : crawling_boxoffice(date_keyword[3], 'movieNm', 'table'),
        "개봉일" : crawling_boxoffice(date_keyword[3], 'openDt', 'table'), "일일 관객수" : crawling_boxoffice(date_keyword[3], 'audiCnt', 'table'),
        "누적 관객수" : crawling_boxoffice(date_keyword[3], 'audiAcc', 'table'), "기준 날짜" : date_list[3]}

table_data5 = {"순위" : crawling_boxoffice(date_keyword[4], 'rank', 'table'), "영화명" : crawling_boxoffice(date_keyword[4], 'movieNm', 'table'),
        "개봉일" : crawling_boxoffice(date_keyword[4], 'openDt', 'table'), "일일 관객수" : crawling_boxoffice(date_keyword[4], 'audiCnt', 'table'),
        "누적 관객수" : crawling_boxoffice(date_keyword[4], 'audiAcc', 'table'), "기준 날짜" : date_list[4]}

table1 = pd.DataFrame(table_data1) # 5일전 기준 박스오피스
table2 = pd.DataFrame(table_data2) # 4일전 기준 박스오피스
table3 = pd.DataFrame(table_data3) # 3일전 기준 박스오피스
table4 = pd.DataFrame(table_data4) # 2일전 기준 박스오피스
table5 = pd.DataFrame(table_data5) # 어제 박스오피스 

table = pd.concat([table1, table2, table3, table4, table5])
table

table_number1 = crawling_boxoffice(date_keyword[4], 'movieNm', 'table')[0] # 어제 기준 박스오피스 1위 작품
final_table1 = table[table['영화명'] == table_number1]

table_number2 = crawling_boxoffice(date_keyword[4], 'movieNm', 'table')[1] # 어제 기준 박스오피스 2위 작품
final_table2 = table[table['영화명'] == table_number2]

table_number3 = crawling_boxoffice(date_keyword[4], 'movieNm', 'table')[2] # 어제 기준 박스오피스 3위 작품
final_table3 = table[table['영화명'] == table_number3]

table_number4 = crawling_boxoffice(date_keyword[4], 'movieNm', 'table')[3] # 어제 기준 박스오피스 4위 작품
final_table4 = table[table['영화명'] == table_number4]

table_number5 = crawling_boxoffice(date_keyword[4], 'movieNm', 'table')[4] # 어제 기준 박스오피스 5위 작품
final_table5 = table[table['영화명'] == table_number5]

In [18]:
# 그래프 생성용 (관객수 데이터 int)
graph_data1 = {"순위" : crawling_boxoffice(date_keyword[0], 'rank', 'graph'), "영화명" : crawling_boxoffice(date_keyword[0], 'movieNm', 'graph'),
        "개봉일" : crawling_boxoffice(date_keyword[0], 'openDt', 'graph'), "일일 관객수" : crawling_boxoffice(date_keyword[0], 'audiCnt', 'graph'),
        "누적 관객수" : crawling_boxoffice(date_keyword[0], 'audiAcc', 'graph'), "기준 날짜" : date_list[0]}

graph_data2 = {"순위" : crawling_boxoffice(date_keyword[1], 'rank', 'graph'), "영화명" : crawling_boxoffice(date_keyword[1], 'movieNm', 'graph'),
        "개봉일" : crawling_boxoffice(date_keyword[1], 'openDt', 'graph'), "일일 관객수" : crawling_boxoffice(date_keyword[1], 'audiCnt', 'graph'),
        "누적 관객수" : crawling_boxoffice(date_keyword[1], 'audiAcc', 'graph'), "기준 날짜" : date_list[1]}

graph_data3 = {"순위" : crawling_boxoffice(date_keyword[2], 'rank', 'graph'), "영화명" : crawling_boxoffice(date_keyword[2], 'movieNm', 'graph'),
        "개봉일" : crawling_boxoffice(date_keyword[2], 'openDt', 'graph'), "일일 관객수" : crawling_boxoffice(date_keyword[2], 'audiCnt', 'graph'),
        "누적 관객수" : crawling_boxoffice(date_keyword[2], 'audiAcc', 'graph'), "기준 날짜" : date_list[2]}

graph_data4 = {"순위" : crawling_boxoffice(date_keyword[3], 'rank', 'graph'), "영화명" : crawling_boxoffice(date_keyword[3], 'movieNm', 'graph'),
        "개봉일" : crawling_boxoffice(date_keyword[3], 'openDt', 'graph'), "일일 관객수" : crawling_boxoffice(date_keyword[3], 'audiCnt', 'graph'),
        "누적 관객수" : crawling_boxoffice(date_keyword[3], 'audiAcc', 'graph'), "기준 날짜" : date_list[3]}

graph_data5 = {"순위" : crawling_boxoffice(date_keyword[4], 'rank', 'graph'), "영화명" : crawling_boxoffice(date_keyword[4], 'movieNm', 'graph'),
        "개봉일" : crawling_boxoffice(date_keyword[4], 'openDt', 'graph'), "일일 관객수" : crawling_boxoffice(date_keyword[4], 'audiCnt', 'graph'),
        "누적 관객수" : crawling_boxoffice(date_keyword[4], 'audiAcc', 'graph'), "기준 날짜" : date_list[4]}

graph_table1 = pd.DataFrame(graph_data1) # 5일전 기준 박스오피스
graph_table2 = pd.DataFrame(graph_data2) # 4일전 기준 박스오피스
graph_table3 = pd.DataFrame(graph_data3) # 3일전 기준 박스오피스
graph_table4 = pd.DataFrame(graph_data4) # 2일전 기준 박스오피스
graph_table5 = pd.DataFrame(graph_data5) # 어제 박스오피스 

graph_table = pd.concat([graph_table1, graph_table2, graph_table3, graph_table4, graph_table5])
graph_table

table_number1 = crawling_boxoffice(date_keyword[4], 'movieNm', 'graph')[0] # 어제 기준 박스오피스 1위 작품
final_graph_table1 = graph_table[graph_table['영화명'] == table_number1]

table_number2 = crawling_boxoffice(date_keyword[4], 'movieNm', 'graph')[1] # 어제 기준 박스오피스 2위 작품
final_graph_table2 = graph_table[graph_table['영화명'] == table_number2]

table_number3 = crawling_boxoffice(date_keyword[4], 'movieNm', 'graph')[2] # 어제 기준 박스오피스 3위 작품
final_graph_table3 = graph_table[graph_table['영화명'] == table_number3]

table_number4 = crawling_boxoffice(date_keyword[4], 'movieNm', 'graph')[3] # 어제 기준 박스오피스 4위 작품
final_graph_table4 = graph_table[graph_table['영화명'] == table_number4]

table_number5 = crawling_boxoffice(date_keyword[4], 'movieNm', 'graph')[4] # 어제 기준 박스오피스 5위 작품
final_graph_table5 = graph_table[graph_table['영화명'] == table_number5]

In [19]:
def index_control(table):
  table.index = table['기준 날짜']
  table.drop(['기준 날짜'], axis=1, inplace=True)
  return table

def make_graph(table, num, movie_name):
  pio.templates.default = "plotly_dark"
  graph = go.Scatter(x=table['기준 날짜'], y=table['일일 관객수'], line={'color':'red','width':2})
  layout = go.Layout(title='어제 기준 국내 박스오피스 '+ num +'위 영화 관객수 추이' +' (' +  movie_name + ')', font={'family':'Malgun Gothic', 'size': 12},
                    xaxis={'title':'날짜'},yaxis={'title':'관객수'},width=1000,height=700)
  fig = go.Figure(data=graph, layout=layout)
  fig.update_xaxes(dtick='D1', tickformat='%Y %m %d')
  fig.update_yaxes(tickformat=',')
  fig.show()

In [20]:
# 어제 국내 박스오피스 순위
table5.index = table5['순위']
table5.drop(['순위'], axis=1, inplace=True)
table5

,영화명,개봉일,일일 관객수,누적 관객수,기준 날짜
순위,,,,,
1위,공조2: 인터내셔날,2022-09-07,"73,681명","6,418,975명",2022-10-08
2위,인생은 아름다워,2022-09-28,"54,635명","499,618명",2022-10-08
3위,극장판 짱구는 못말려: 수수께끼! 꽃피는 천하떡잎학교,2022-09-28,"49,650명","327,430명",2022-10-08
4위,정직한 후보2,2022-09-28,"44,676명","633,560명",2022-10-08
5위,스마일,2022-10-06,"12,834명","32,499명",2022-10-08
6위,극장판 헬로카봇: 수상한 마술단의 비밀,2022-09-28,"11,913명","162,472명",2022-10-08
7위,고고다이노 극장판: 얼음공룡의 비밀,2022-10-05,"10,069명","17,102명",2022-10-08
8위,컴백홈,2022-10-05,"5,473명","30,903명",2022-10-08
9위,스파이더맨: 노 웨이 홈 '펀버전',2022-10-05,"4,782명","14,957명",2022-10-08


In [21]:
# 어제 박스오피스 1위 작품 관객수 추이
index_control(final_table1)

,순위,영화명,개봉일,일일 관객수,누적 관객수
기준 날짜,,,,,
2022-10-04,1위,공조2: 인터내셔날,2022-09-07,"32,755명","6,246,576명"
2022-10-05,1위,공조2: 인터내셔날,2022-09-07,"27,528명","6,274,104명"
2022-10-06,1위,공조2: 인터내셔날,2022-09-07,"28,024명","6,302,128명"
2022-10-07,1위,공조2: 인터내셔날,2022-09-07,"43,166명","6,345,294명"
2022-10-08,1위,공조2: 인터내셔날,2022-09-07,"73,681명","6,418,975명"


In [22]:
make_graph(final_graph_table1, '1', table_number1)

In [23]:
# 어제 박스오피스 2위 작품 관객수 추이
index_control(final_table2)

,순위,영화명,개봉일,일일 관객수,누적 관객수
기준 날짜,,,,,
2022-10-04,3위,인생은 아름다워,2022-09-28,"25,258명","363,310명"
2022-10-05,2위,인생은 아름다워,2022-09-28,"24,760명","388,070명"
2022-10-06,2위,인생은 아름다워,2022-09-28,"24,418명","412,488명"
2022-10-07,2위,인생은 아름다워,2022-09-28,"32,495명","444,983명"
2022-10-08,2위,인생은 아름다워,2022-09-28,"54,635명","499,618명"


In [24]:
make_graph(final_graph_table2, '2', table_number2)

In [25]:
# 어제 박스오피스 3위 작품 관객수 추이
index_control(final_table3)

,순위,영화명,개봉일,일일 관객수,누적 관객수
기준 날짜,,,,,
2022-10-04,4위,극장판 짱구는 못말려: 수수께끼! 꽃피는 천하떡잎학교,2022-09-28,"9,717명","241,146명"
2022-10-05,4위,극장판 짱구는 못말려: 수수께끼! 꽃피는 천하떡잎학교,2022-09-28,"9,949명","251,095명"
2022-10-06,4위,극장판 짱구는 못말려: 수수께끼! 꽃피는 천하떡잎학교,2022-09-28,"9,962명","261,057명"
2022-10-07,4위,극장판 짱구는 못말려: 수수께끼! 꽃피는 천하떡잎학교,2022-09-28,"16,723명","277,780명"
2022-10-08,3위,극장판 짱구는 못말려: 수수께끼! 꽃피는 천하떡잎학교,2022-09-28,"49,650명","327,430명"


In [26]:
make_graph(final_graph_table3, '3', table_number3)

In [12]:
# 어제 박스오피스 4위 작품 관객수 추이
index_control(final_table4)

,순위,영화명,개봉일,일일 관객수,누적 관객수
기준 날짜,,,,,
2022-10-04,2위,정직한 후보2,2022-09-28,"25,449명","518,257명"
2022-10-05,3위,정직한 후보2,2022-09-28,"20,128명","538,385명"
2022-10-06,3위,정직한 후보2,2022-09-28,"21,232명","559,617명"
2022-10-07,3위,정직한 후보2,2022-09-28,"29,267명","588,884명"
2022-10-08,4위,정직한 후보2,2022-09-28,"44,676명","633,560명"


In [27]:
make_graph(final_graph_table4, '4', table_number4)

In [28]:
# 어제 박스오피스 5위 작품 관객수 추이
index_control(final_table5)

,순위,영화명,개봉일,일일 관객수,누적 관객수
기준 날짜,,,,,
2022-10-06,5위,스마일,2022-10-06,"9,402명","9,677명"
2022-10-07,5위,스마일,2022-10-06,"9,988명","19,665명"
2022-10-08,5위,스마일,2022-10-06,"12,834명","32,499명"


In [29]:
make_graph(final_graph_table5, '5', table_number5)